In [2]:
import utils
import math
import re
import itertools
import matplotlib.pyplot as plt
from typing import NamedTuple
from collections import defaultdict, deque

## Day 7: Handy Haversacks 

[#](https://adventofcode.com/2020/day/7). We a list of rules laying out what color bags and their number can be in a particular colored bag.

We want to know how many colors can a `shiny gold` bag be in. Now this can be a bag within a bag within a bag... so it looks like a recursion or some such problem?

In [3]:
test7 = """light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.""".splitlines()

inp7 = utils.get_input(7, splitlines=True)

First up, a simple function to parse an individual bag:

In [4]:
def parse_bag(txt):
    if txt == "no other bags":
        return "FULL", 0
    num, col1, col2, _ = txt.split(" ")
    return f"{col1} {col2}", int(num)

parse_bag('2 muted yellow bags'), parse_bag("no other bags")

(('muted yellow', 2), ('FULL', 0))

In [5]:
def parse_7(inp=test7, verbose=False):

    bags = {}

    for line in inp:
        color, rr = line.split(" bags contain ")
        bag_rules = rr.strip(".").split(", ")
        
        rules_dict = {}
        for bag_rule in bag_rules:
            col, num = parse_bag(bag_rule)
            rules_dict[col] = num

        if verbose:
            print(bag_rules)
            print(rules_dict)
            print("----")
        bags[color] = rules_dict
        
    return bags

rules = parse_7()
rules

{'light red': {'bright white': 1, 'muted yellow': 2},
 'dark orange': {'bright white': 3, 'muted yellow': 4},
 'bright white': {'shiny gold': 1},
 'muted yellow': {'shiny gold': 2, 'faded blue': 9},
 'shiny gold': {'dark olive': 1, 'vibrant plum': 2},
 'dark olive': {'faded blue': 3, 'dotted black': 4},
 'vibrant plum': {'faded blue': 5, 'dotted black': 6},
 'faded blue': {'FULL': 0},
 'dotted black': {'FULL': 0}}

In [18]:
find = "shiny gold"
stack = list(rules[find].keys()) # first level bags
colors = stack[:]
print(colors)

for col in stack:
    if cc := list(rules[col].keys()):
        print(col, cc)
        colors.append(cc)
        for c in cc:
            stack.append(cc)
        stack = stack[1:]

['dark olive', 'vibrant plum']
dark olive ['faded blue', 'dotted black']
vibrant plum ['faded blue', 'dotted black']


TypeError: unhashable type: 'list'

In [17]:
colors

['dark olive',
 'vibrant plum',
 dict_keys(['faded blue', 'dotted black']),
 dict_keys(['faded blue', 'dotted black'])]

In [ ]:
find="shiny gold"
ans = list(rules[find].keys())

def rr(col):
    ans = list(rules[find].keys())
    
    

In [121]:
def solve_7(find="shiny gold", inp=test7):
    rules = parse_7(inp)
    
solve_7()

{'light red': {'bright white': 1, 'muted yellow': 2}, 'dark orange': {'bright white': 3, 'muted yellow': 4}, 'bright white': {'shiny gold': 1}, 'muted yellow': {'shiny gold': 2, 'faded blue': 9}, 'shiny gold': {'dark olive': 1, 'vibrant plum': 2}, 'dark olive': {'faded blue': 3, 'dotted black': 4}, 'vibrant plum': {'faded blue': 5, 'dotted black': 6}, 'faded blue': {'FULL': 0}, 'dotted black': {'FULL': 0}}


## Part 2

Now we need to validate data ranges. We already have a list of passports so I'll reuse that.

First up writing some helper functions to validate specific passport fields: